Own code

In [ ]:
import sys
# sys.path.append("../code")
# all file locations are relative to the notebook on my local file system

# Get the Node and Workflow object
from nipype import Node, Workflow
import import nipype.interfaces.spm as spm
from nipype.interfaces.matlab import MatlabCommand

# Specify which SPM to use
MatlabCommand.set_default_paths('../../../MATLAB/spm12')

# Location of sample data
sample_dir = '../sample_data'

# # What is the name of your working directory and output folder
# output_dir =  'output_firstSteps'
# working_dir = 'workingdir_firstSteps'

# What are experiment specific parameters
# number_of_slices = 40
time_repetition = 2.0
fwhm = 6

Example from Nipype tutorials: https://miykael.github.io/nipype-beginner-s-guide/nipype.html#a-short-example

In [ ]:
#Import modules
import nipype
import nipype.interfaces.afni        as afni
import nipype.interfaces.freesurfer  as fs
import nipype.interfaces.ants        as ants
import nipype.interfaces.fsl         as fsl
import nipype.interfaces.nipy        as nipy
import nipype.interfaces.spm         as spm


#Specify experiment specifc parameters
experiment_dir = '~/experiment_folder'
nameofsubjects = ['subject1','subject2','subject3']

#Where can the raw data be found?
grabber = nipype.DataGrabber()
grabber.inputs.base_directory = experiment_dir + '/data'
grabber.inputs.subject_id = nameofsubjects

#Where should the output data be stored at?
sink = nipype.DataSink()
sink.inputs.base_directory = experiment_dir + '/output_folder'

#Create a node for each step of the analysis

#Motion Correction (AFNI)
realign = afni.Retroicor()

#Create a workflow to connect all those nodes
analysisflow = nipype.Workflow()

#Connect the nodes to each other
analysisflow.connect([(grabber     ->  realign    ),
                      (realign     ->  coreg      ),
                      (coreg       ->  normalize  ),
                      (normalize   ->  smooth     ),
                      (smooth      ->  modelspec  ),
                      (modelspec   ->  modelest   ),
                      (modelest    ->  contrastest),
                      (contrastest ->  threshold  ),
                      (threshold   ->  sink       )
                      ])

#Run the workflow in parallel
analysisflow.run(mode='parallel')

Example output from giraffe

In [ ]:
import sys
import nipype
import nipype.pipeline as pe

import nipype.interfaces.spm as spm

#Use spm_realign for estimating within modality rigid body alignment
spm_realign = pe.Node(interface = spm.Realign(), name='spm_realign')
spm_realign.inputs.in_files = fanon-0007_00001.nii, fanon-0007_00002.nii
spm_realign.inputs.jobtype = est_write

#Use spm_smooth for 3D Gaussian smoothing of image volumes.
spm_smooth = pe.Node(interface = spm.Smooth(), name='spm_smooth')

#Generic datasink module to store structured outputs
io_data_sink = pe.Node(interface = io.DataSink(), name='io_data_sink')

#Generic datagrabber module that wraps around glob in an
io_data_grabber = pe.Node(io.DataGrabber(), name = 'io_data_grabber')

#Create a workflow to connect all those nodes
analysisflow = nipype.Workflow('MyWorkflow')
analysisflow.connect(spm_realign, "realigned_files", spm_smooth, "in_files")
